# **Libraries**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# **Data**

## Import Data

In [2]:
df = pd.read_csv(r"D:\Samuel\Open Datasets\Fraud Detection using AI-ML\AIML Dataset.csv")
df.sample(5)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
2345523,189,TRANSFER,30151.10,C2043199796,1222.0,0.0,C1827624480,0.00,30151.10,0,0
6267564,617,PAYMENT,6725.61,C1016871671,0.0,0.0,M16882191,0.00,0.00,0,0
4162204,304,CASH_OUT,103012.08,C1468979835,0.0,0.0,C1434052156,485718.66,588730.74,0,0
1599927,156,PAYMENT,15414.71,C511116464,0.0,0.0,M1368531885,0.00,0.00,0,0
4600522,329,CASH_OUT,12296.86,C1669278560,0.0,0.0,C935764423,75558.34,87855.20,0,0


## Data Exploration

In [3]:
print(df.shape)
df.info()

(6362620, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


In [4]:
df.columns

Index(['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
       'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud',
       'isFlaggedFraud'],
      dtype='object')

In [5]:
df['isFraud'].value_counts()

isFraud
0    6354407
1       8213
Name: count, dtype: int64

In [6]:
df['isFlaggedFraud'].value_counts()

isFlaggedFraud
0    6362604
1         16
Name: count, dtype: int64

In [7]:
df.isnull().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

In [11]:
df.shape[0]

6362620

In [13]:
round((df['isFraud'].value_counts()[1] / df.shape[0]) * 100, 2)

np.float64(0.13)